# 回帰モデル

1. 
2. 
3. 
4. 
5. 
6. 


In [ ]:
import lightgbm as lgb
from sklearn.model_selection import KFold

In [ ]:
params = {
    'objective': 'regression',
    'metric': 'rmse',
    'learning_rate': 0.01,
    'max_depth': -1,
    'num_leaves': 255,
    'max_bin': 255,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'nthread': -1,
    'bagging_freq': 1,
    'verbose': -1,
    'seed': 1,
}

In [ ]:
def lightgbms(X, X_test, y, params, log='non'):

    NFOLDS = 5
    folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=12)

    preds = pd.DataFrame()
    train_preds = pd.DataFrame()

    columns = X.columns
    splits = folds.split(X, y)

    y_preds = np.zeros(X_test.shape[0])
    y_oof = np.zeros(X.shape[0])

    score = 0
    scores = []

    feature_importances = pd.DataFrame()
    feature_importances['feature'] = columns

    for fold_n, (train_index, val_index) in enumerate(splits):
        # split train, Valid
        X_train, X_val = X[columns].iloc[train_index], X[columns].iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        '''
        LightGBM
        '''
        dtrain = lgb.Dataset(X_train, y_train)
        dval = lgb.Dataset(X_val, y_val)

        clf = lgb.train(params,
                        dtrain,
                        1000,
                        valid_sets=[dtrain, dval],
                        categorical_feature=col_category,
                        verbose_eval=200,
                        early_stopping_rounds=500)

        feature_importances[f'fold_{fold_n + 1}'] = clf.feature_importance()

        '''
        LightGBM
        '''
        # stack
        y_pred_val = clf.predict(X_val)
        y_oof[val_index] = y_pred_val
        
        
        #print(f"Fold {fold_n + 1} | rmse: {np.sqrt(mean_squared_error(np.exp(y_val),np.exp( y_pred_val)))}")
        score += np.sqrt(mean_squared_error(np.exp(y_val), np.exp(y_pred_val))) / NFOLDS
        scores.append(np.sqrt(mean_squared_error(np.exp(y_val), np.exp(y_pred_val))))

        #print(f"Fold {fold_n + 1} | rmse: {np.sqrt(mean_squared_error(y_val, y_pred_val))}")

        y_preds += clf.predict(X_test) / NFOLDS

        train_preds[f"Fold {fold_n + 1}"] = clf.predict(X)

        preds[f"Fold {fold_n + 1}"] = clf.predict(X_test)

        del X_train, X_val, y_train, y_val
        gc.collect()
    #print(f"\nMean RSME = {score}")

    return y_oof, y_preds, feature_importances, scores



# ディープラーニング

In [ ]:
def build_model(df):
    model = keras.Sequential([
        keras.layers.Dense(64, activation=tf.nn.relu,
                           input_shape=(df.shape[1],)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(128, activation=tf.nn.relu),
        keras.layers.Dense(128, activation=tf.nn.relu),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(64, activation=tf.nn.relu),
        keras.layers.Dense(1)
    ])
    
    optimizer = tf.train.RMSPropOptimizer(0.001)
    
    model.compile(loss='mse',
                  optimizer=optimizer,
                  metrics=['mse'])
    return model

In [ ]:
def KDeepLearning(X, X_test, y):
    
    EPOCHS = 500
    NFOLDS = 5
    
    folds = KFold(n_splits=NFOLDS, shuffle=True, random_state=1)

    preds = pd.DataFrame()
    train_preds = pd.DataFrame()

    columns = X.columns
    splits = folds.split(X, y)

    y_preds = np.zeros(X_test.shape[0])
    y_oof = np.zeros(X.shape[0])
    
    score = 0
    scores = []

    for fold_n, (train_index, val_index) in enumerate(splits):
        # split train, Valid
        X_train, X_val = X[columns].iloc[train_index], X[columns].iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        '''
        DeepLearning
        '''
        
        early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
        model = build_model(X)
        
        history = model.fit(X_train, y_train, 
                            epochs=500, 
                            batch_size=100, 
                            verbose=0, 
                            validation_data=(X_val, y_val),
                            callbacks=[early_stop])
        '''
        DeepLearning
        '''
        
        # stack
        y_pred_val = model.predict(X_val).flatten()
        y_oof[val_index] = y_pred_val
        
        print(f"Fold {fold_n + 1} | rmse: {np.sqrt(mean_squared_error(np.exp(y_val),np.exp( y_pred_val)))}")
        
        #Score
        score += np.sqrt(mean_squared_error(np.exp(y_val), np.exp(y_pred_val))) / NFOLDS
        scores.append(np.sqrt(mean_squared_error(np.exp(y_val), np.exp(y_pred_val))))
        
        
        y_preds += model.predict(X_test).flatten() / NFOLDS

        train_preds[f"Fold {fold_n + 1}"] = model.predict(X).flatten()

        preds[f"Fold {fold_n + 1}"] = model.predict(X_test).flatten()

        del X_train, X_val, y_train, y_val
        gc.collect()
        
    print(f"\nMean RSME = {score}")

    return y_oof, y_preds, scores

